## Overview

This notebook shows how to use Instructor with LLMs to generate simple Python code. It uses validation to ensure the dependencies are correct and that the code runs without errors.

Note: this notebook assumes you're using Google Colab. You can safely edit / play here. Or go to `File` -> `Save a copy in Google Drive` to make your own version.

In [3]:
%%capture
!pip install instructor groq openai

On the left, click the key and set two secrets with your keys. Be sure to enable "Notebook access" for them. This is how Google Colab works...you're not sharing your keys with anyone.

OPENAI_API_KEY

GROQ_API_KEY

In [5]:
import instructor
import openai
import groq
from pydantic import BaseModel, Field, field_validator
from typing import List
import subprocess
from rich import print as rprint
import os

try:
    from google.colab import userdata
    os.environ['OPENAI_API_KEY'] = '' or userdata.get('OPENAI_API_KEY') # or put your key in the '' on this line
    os.environ['GROQ_API_KEY'] = '' or userdata.get('GROQ_API_KEY')
except Exception as e:
    # print(e)
    pass


Now to the cool stuff...

In [ ]:
inference_provider = "openai"   # "openai" or "groq"
client = instructor.from_openai(openai.OpenAI()) if inference_provider == "openai" else instructor.from_groq(groq.Groq())

class VerifiedPythonCodeAndDependencies(BaseModel):
    """
    Python code and dependencies to meet specified requirements. Dependencies will be installed and the code will be executed.

    Limitations:
    - This code will not be tested for quality, only that it runs without errors.
    """
    architecture: str = Field(description="A markdown file that describes the architecture of the code and how it will satisfy each requirement one by one.")
    dependencies: List[str] = Field(description="List of pip packages required to run the code.")
    code: str = Field(description="A single Python file code.")

    # the order above specifies the order of validation

    @field_validator('dependencies')
    @classmethod
    def validate_dependencies(cls, v):
        if v:
            try:
                subprocess.run(["pip", "install"] + v, check=True)
                print("Dependencies installed: " + ", ".join(v))
                print("\n\n")
            except subprocess.CalledProcessError as e:
                error_value = "Failed to install dependencies: " + str(e)
                print(error_value)
                raise ValueError(error_value)
        return v

    @field_validator('code')
    @classmethod
    def validate_code(cls, v):
        if v:
            try:
                print("Python code\n===========\n")
                rprint(code)
                print("\n\nTrying code\n===========\n")
                exec(code)
            except subprocess.CalledProcessError as e:
                error_value = "Error while executing code: " + str(e)
                print(error_value)
                raise ValueError(error_value)
        return v

result = client.chat.completions.create(
    model="llama3-70b-8192" if inference_provider == "groq" else "gpt-4-turbo",
    response_model=VerifiedPythonCodeAndDependencies,
    temperature=0.0,
    max_retries=3,
    messages=[{"role": "user", "content": """
        Extract the text content of https://example.com as markdown and print it out.
        """}]
    )

code = result.code
# rprint(code)